# BOJ Helper

In [ ]:
import os
import subprocess
import time
import webbrowser
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyperclip
from IPython.display import display, HTML
import ipywidgets as widgets

<!-- margin -->

In [ ]:
problem_id = 1000

In [ ]:
copy_source = True
automatic_backup = True
make_description = True

In [ ]:
problem_url = f'https://www.acmicpc.net/problem/{problem_id}'
submit_url = f'https://www.acmicpc.net/submit/{problem_id}'

In [ ]:
source_path = '.\\Project\\BOJ\\Source.cpp'
backup_path = {
    'source': f'.\\Solved\\Sources\\{problem_id}\\{problem_id}.cpp',
    'description': f'.\\Solved\\Sources\\{problem_id}\\README.md',
    'build': f'.\\Solved\\Builds\\{problem_id}.exe',
}
description_path = '.\\Temp\\README.md'
build_path = '.\\Temp\\BOJ.exe'

In [ ]:
build_command = f'g++ "{source_path}" -o "{build_path}" -O2 -Wall -lm -std=gnu++17'
backup_command = {
    'source': f'''echo F | xcopy "{source_path}" "{backup_path['source']}" /Y''',
    'description': f'''echo F | xcopy "{description_path}" "{backup_path['description']}" /Y''',
    'build': f'''echo F | xcopy "{build_path}" "{backup_path['build']}" /Y''',
}

<!-- margin -->

In [ ]:
response = requests.get(problem_url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
else:
    raise Exception(f'status code: {response.status_code}')

<!-- margin -->

In [ ]:
if os.system(build_command):
    raise Exception('compile error')

In [ ]:
sample_inputs = soup.select('pre[id^="sample-input-"]')
sample_outputs = soup.select('pre[id^="sample-output-"]')

time_limit = float(soup.select_one('#problem-info td:nth-child(1)').text.split()[0])
memory_limit = float(soup.select_one('#problem-info td:nth-child(2)').text.split()[0])

result = pd.DataFrame(columns=['정오', '출력', '정답', '시간'])

for sample_input, sample_output in zip(sample_inputs, sample_outputs):
    process = subprocess.Popen([build_path], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    time.sleep(1e-2)

    start_time = time.time()
    output, error = process.communicate(sample_input.text.encode(), timeout=time_limit)
    end_time = time.time()

    output = output.decode().replace('\r\n', '\n').rstrip('\n')
    sample_output = sample_output.text.replace('\r\n', '\n').rstrip('\n')
    corrected = output == sample_output
    elapsed_time = (end_time - start_time) * 1000

    result = result.append({
        '정오': corrected,
        '출력': output,
        '정답': sample_output,
        '시간': elapsed_time,
    }, ignore_index=True)

accuracy = result['정오'].mean() * 100
mean_time = result['시간'].mean()

result['정오'] = result['정오'].apply(lambda x : ('❌', '✔️')[x])
result['시간'] = result['시간'].apply(lambda x : f'{x:.1f}ms')

result = result.append({
    '정오': f'{accuracy:.1f}%',
    '출력': '-',
    '정답': '-',
    '시간': f'{mean_time:.1f}ms',
}, ignore_index=True)

<!-- margin -->

In [ ]:
if copy_source:
    with open(source_path) as source:
        pyperclip.copy(source.read())

In [ ]:
if automatic_backup:
    os.system(backup_command['source'])
    os.system(backup_command['build'])

In [ ]:
if make_description:
    with open(description_path, 'w', encoding='utf-8') as description:
        problem_title = soup.select_one('#problem_title').text
        problem_sections = soup.select('.problem-section:not([style])')

        description.write(f'# {problem_title}\n\n')

        for problem_section in problem_sections:
            headline = problem_section.select_one('.headline').text.strip().replace('\n', '  \n')
            problem_text = problem_section.select_one('.problem-text').decode_contents().strip().replace('\n', '  \n')

            description.write(f'## {headline}\n')
            description.write(f'{problem_text}\n\n')

    os.system(backup_command['description'])

<!-- margin -->

In [ ]:
table = result.to_html().replace('\\n','<br>')
with open('table.css') as css:
    style = css.read()

html = f'<style>{style}</style> {table}'
display(HTML(html))

if accuracy == 100:
    button = widgets.Button(description='제출 페이지 열기', button_style='success')
    button.on_click(lambda x : webbrowser.open(submit_url))
    display(button)

<!-- margin -->